In [1]:
import pandas as pd

### 董事監 data

In [5]:
df = pd.read_excel('董事監名單.xlsx')

In [6]:
df.head()

,公司名稱,公司代碼,市場類別,產業類別,職稱,姓名,所代表法人姓名
0,台泥,1101,上市,水泥工業,董事長之法人代表人,張安平,嘉利實業股份有限公司
1,台泥,1101,上市,水泥工業,董事之法人代表人,余俊彥,中成開發投資股份有限公司
2,台泥,1101,上市,水泥工業,董事之法人代表人,辜公怡,泰和興業股份有限公司
3,台泥,1101,上市,水泥工業,董事之法人代表人,李鐘培,財團法人辜公亮文教基金會
4,台泥,1101,上市,水泥工業,董事之法人代表人,王伯元,富品投資股份有限公司


In [7]:
df['姓名'].value_counts().head()

侯榮顯    11
莊永順    11
魏啟林    10
王文淵    10
許介立     9
Name: 姓名, dtype: int64

In [8]:
df['所代表法人姓名'].value_counts().head()

財政部             54
中國鋼鐵股份有限公司      42
行政院國家發展基金管理會    35
統一企業股份有限公司      28
經濟部             26
Name: 所代表法人姓名, dtype: int64

### 遺失值處理

In [10]:
df['所代表法人姓名'] = df['所代表法人姓名'].fillna('無')
df = df.dropna()
df = df.reset_index(drop=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16591 entries, 0 to 16590
Data columns (total 7 columns):
公司名稱       16591 non-null object
公司代碼       16591 non-null int64
市場類別       16591 non-null object
產業類別       16591 non-null object
職稱         16591 non-null object
姓名         16591 non-null object
所代表法人姓名    16591 non-null object
dtypes: int64(1), object(6)
memory usage: 907.4+ KB


In [12]:
df.to_csv("董事監名單_clean.csv",index=False)

In [13]:
df.to_excel("董事監名單_clean.xlsx",index=False)

### 董事監 clean data

In [167]:
df = pd.read_excel('董事監名單_clean.xlsx')

In [4]:
df.head(2)

,公司名稱,公司代碼,市場類別,產業類別,職稱,姓名,所代表法人姓名
0,台泥,1101,上市,水泥工業,董事長之法人代表人,張安平,嘉利實業股份有限公司
1,台泥,1101,上市,水泥工業,董事之法人代表人,余俊彥,中成開發投資股份有限公司


In [11]:
# pd.DataFrame(df['所代表法人姓名'].value_counts()).to_excel("所代表法人姓名.xlsx")

In [4]:
# 爬google
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm_notebook
com = []
for i in df['所代表法人姓名'].value_counts().index:
    com.append(i)
com_ne, ID = [], []
driver = webdriver.Chrome(executable_path='/Users/andy/anaconda/selenium/chromedriver')
for name in tqdm_notebook(com[1961:]):
    driver.get("https://www.google.com.tw/")
    driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input').send_keys(name+' 股票')
    driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input').send_keys(Keys.ENTER)
    try:
        ID.append(driver.find_element_by_xpath('//*[@id="knowledge-finance-wholepage__entity-summary"]/div/g-card-section/div/g-card-section/div[1]/span').text)
        com_ne.append(name)
        time.sleep(1)
    except:
        time.sleep(1)
        pass
pd.DataFrame(list(zip(com_ne, ID))).to_excel("轉換公司2.xlsx", index=False)

In [171]:
df[df['公司代碼']==2104].head(2)

,公司名稱,公司代碼,市場類別,產業類別,職稱,姓名,所代表法人姓名
2932,國際中橡,2104,上市,橡膠工業,獨立董事本人,池慶康,無
2933,國際中橡,2104,上市,橡膠工業,董事之法人代表人,辜公愷,富品投資股份有限公司


In [33]:
df2 = pd.read_excel('轉換公司.xlsx')

In [40]:
df2['num'] = df2[1].str.split(':').str[1]

In [153]:
df2['aaa'] = df2[1].str.split(':').str[0]

In [154]:
a,b = [],[]
for i in range(len(df2['num'])):
    if df2['aaa'][i] == 'TPE':
        try:
            a.append(df[df['公司代碼']==int(df2['num'][i][1:5])].head(1)['公司名稱'].values)
            b.append(df2[0][i])
        except:
            pass

In [157]:
c = []
for i in a:
    for k in i:
        c.append(k)

In [160]:
new = list(zip(b,c))

In [168]:
for i in new:
    df['所代表法人姓名'] = df['所代表法人姓名'].replace(i[0],i[1])

In [173]:
df.to_csv('董事監名單_clean2.csv', index=False)